In [ ]:
%load_ext autoreload
%autoreload 2

import argparse
import logging
import time
from os.path import join
import sys
sys.path.append('../')
import warnings
warnings.filterwarnings("ignore")

from coffea import processor, util

In [ ]:
from collections import defaultdict
import os
import hist
import mplhep as hep
import seaborn as sns
from coffea import util
from matplotlib import pyplot as plt
from cycler import cycler
from hist import Hist
from hist.intervals import ratio_uncertainty
from azh_analysis.utils.plotting import plot_variable
import warnings
warnings.filterwarnings('ignore')
hep.style.use(["CMS", "fira", "firamath"])

data = util.load("../data_UL_2018_02-14.coffea")
mc = util.load("../MC_UL_2018_02-14.coffea")

cat_labels = {
    'tt': r'$ll\tau_h\tau_h$',
    'et': r'$ll e\tau_h$',
    'mt': r'$ll\mu\tau_h$',
    'em': r'$ll e\mu$',
}

In [ ]:
var = "mll"
for year in ['2018']:
    for cat, cat_label in cat_labels.items():
        for bcat in [0]:
            mc_s = sum(mc[var].values())
            mc_s = mc_s[:, 'ee'+cat, ::sum, "nom", :] + mc_s[:, 'mm'+cat, ::sum, "nom", :]
            data_s = sum(data[var].values())
            data_s = data_s[:, 'ee'+cat, ::sum, "none", : ] + data_s[:, 'mm'+cat, ::sum, "none", : ]
            plot_variable(data_s, mc_s, "mll", cat_label=cat_label, var_label=r"$m_{ll}$")

In [ ]:
var = "mtt"
for year in ['2018']:
    for cat, cat_label in cat_labels.items():
        for bcat in [0]:
            for mass_type in ["corr", "cons"]:
                mc_s = sum(mc[var].values())
                mc_s = mc_s[:, 'ee'+cat, mass_type, ::sum, "nom", :] + mc_s[:, 'mm'+cat, mass_type, ::sum, "nom", :]
                data_s = sum(data[var].values())
                data_s = data_s[:, 'ee'+cat, mass_type, ::sum, "none", : ] + data_s[:, 'mm'+cat, mass_type, ::sum, "none", : ]
                var_label = r"$m_{\tau\tau}^\mathrm{corr}$ [GeV]" if mass_type=="corr" else r"$m_{\tau\tau}^\mathrm{cons}$ [GeV]"
                plot_variable(data_s, mc_s, "mass", cat_label=cat_label, var_label=var_label)

In [ ]:
var = "m4l"
for year in ['2018']:
    for cat, cat_label in cat_labels.items():
        for bcat in [0]:
            for mass_type in ["raw", "corr", "cons"]:
                mc_s = sum(mc[var].values())
                mc_s = mc_s[:, 'ee'+cat, mass_type, ::sum, "nom", :] + mc_s[:, 'mm'+cat, mass_type, ::sum, "nom", :]
                data_s = sum(data[var].values())
                data_s = data_s[:, 'ee'+cat, mass_type, ::sum, "none", : ] + data_s[:, 'mm'+cat, mass_type, ::sum, "none", : ]
                var_labels = {
                    "corr": r"$m_{ll\tau\tau}^\mathrm{corr}$ [GeV]",
                    "cons": r"$m_{ll\tau\tau}^\mathrm{cons}$ [GeV]",
                    "raw": r"$m_{ll\tau\tau}^\mathrm{raw}$ [GeV]",
                }
                plot_variable(data_s, mc_s, "mass", cat_label=cat_label, var_label=var_labels[mass_type])

In [ ]:
# pt:  group, category, leg, btags, syst_shift, vals
var = "pt"
for year in ['2018']:
    for cat, cat_label in cat_labels.items():
        for bcat in [0]:
            for leg in ["1", "2", "3", "4"]:
                mc_s = sum(mc[var].values())
                mc_s = mc_s[:, 'ee'+cat, leg, ::sum, "nom", :] + mc_s[:, 'mm'+cat, leg, ::sum, "nom", :]
                data_s = sum(data[var].values())
                data_s = data_s[:, 'ee'+cat, leg, ::sum, "none", : ] + data_s[:, 'mm'+cat, leg, ::sum, "none", : ]
                var_label = f"Leg {leg} " + r"$p_T$ [GeV]"
                plot_variable(data_s, mc_s, "pt", cat_label=cat_label, var_label=var_label)

In [ ]:
var = "met"
for year in ['2018']:
    for cat, cat_label in cat_labels.items():
        for bcat in [0]:
            mc_s = sum(mc[var].values())
            mc_s = mc_s[:, 'ee'+cat, ::sum, "nom", :] + mc_s[:, 'mm'+cat, ::sum, "nom", :]
            data_s = sum(data[var].values())
            data_s = data_s[:, 'ee'+cat, ::sum, "none", : ] + data_s[:, 'mm'+cat, ::sum, "none", : ]
            plot_variable(data_s, mc_s, "met", cat_label=cat_label, var_label=r"$E_T^\mathrm{miss}$ [GeV]")